## Random Forest Classifier with Pipeline and Hyperparameter Tuning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Load the dataset

df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
# Check for missing values
df.isnull().sum()


total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [4]:
# Check the data types
df.dtypes

total_bill     float64
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
dtype: object

In [5]:
df.describe() # Summary statistics

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [9]:
df['day'].unique()



['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [8]:
df['sex'].unique()

['Female', 'Male']
Categories (2, object): ['Male', 'Female']

In [10]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

### Here we are Predicting which time customer went to restaurant, like dinner or lunch

In [11]:
### dependent feature 
df.time

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

In [12]:
# converting int o numerical feature using label encoding
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])

In [13]:
df['time'].unique()

array([0, 1])

In [14]:
## dependent feature and independent feature

X = df.drop(labels='time', axis=1) # labels is used to drop the column
y = df.time

In [16]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [17]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer #handling missing values
from sklearn.preprocessing import StandardScaler #scaling the data
from sklearn.preprocessing import OneHotEncoder #encoding the data
from sklearn.compose import ColumnTransformer #transforming the data in column wise manner 


In [21]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [22]:
categ_col = ['sex', 'smoker', 'day']

num_col = ['total_bill', 'tip', 'size']

In [24]:
## Feature engeering automation 
## numerical pipeline for feature scaling

num_pipeline = Pipeline(
    steps = [
    ('imputer', SimpleImputer(strategy='median')), # handling missing values
    ('scaler', StandardScaler()) # scaling the data
])

#Categorical pipeline for encoding the data

categ_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehotencoder', OneHotEncoder())
    ]
)


In [30]:
## combining the numerical and categorical pipeline using column transformer

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_col),
    ('cat', categ_pipeline, categ_col)
]
)
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day'])])

In [26]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC


In [51]:
## model training automation

models = {
    'RandomForest': RandomForestClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'LogisticRegression': LogisticRegression()
    
}

In [52]:
from sklearn.metrics import accuracy_score

In [58]:
def evaluate_model(X_train,y_train,X_test,y_test,models):
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]# getting the model where i is the index of the model
        
        #train the model
        model.fit(X_train,y_train)
        
        #predict the model
        y_test_pred = model.predict(X_test)
        
        #get the accuracy score
        model_accuracy = accuracy_score(y_test,y_test_pred)
        
        #store the accuracy score in the report
        report[list(models.keys())[i]] = model_accuracy #name of the model where i is the index of the model
        
    return report

In [59]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'RandomForest': 0.9591836734693877,
 'DecisionTree': 0.9387755102040817,
 'LogisticRegression': 1.0}

In [35]:
classifier = RandomForestClassifier()

In [34]:
from sklearn.model_selection import RandomizedSearchCV

In [60]:
## Hyperparameter tuning
## RandomizedSearchCV
params = {'max_depth':[3,5,10,None],
          'n_estimators':[10,100,200,300],
          'criterion':['gini','entropy']
          }
          



In [61]:
cv = RandomizedSearchCV(classifier, param_distributions=params,scoring= 'accuracy',cv=5,verbose=3)
cv.fit(X_train,y_train) 

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=None, n_estimators=10;, score=0.974 total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=None, n_estimators=10;, score=0.923 total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=None, n_estimators=10;, score=1.000 total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=None, n_estimators=10;, score=0.897 total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=None, n_estimators=10;, score=0.923 total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.974 total time=   0.6s
[CV 2/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.923 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.974 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.949 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.923 total time=  

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [10, 100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [62]:
cv.best_params_


{'n_estimators': 200, 'max_depth': None, 'criterion': 'gini'}

In [63]:
cv.best_score_

0.9589743589743589

In [64]:
cv.best_estimator_

RandomForestClassifier(n_estimators=200)

In [ ]:
## internal assignment
## Take totall bill as dependent feature and all other features as independent feature
